In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta
from glob import glob
import matplotlib.pyplot as plt

In [2]:
july = pd.read_csv("./집단 분리/월별/analysis/7_analysis.csv")
august = pd.read_csv("./집단 분리/월별/analysis/8_analysis.csv")

In [3]:
# 주말 여부
def weekend(x):
    if x.dayofweek in [5, 6]:
        return 1
    else: 
        return 0

In [6]:
july['create_dt'] = pd.to_datetime(july['create_dt'])
august['create_dt'] = pd.to_datetime(august['create_dt'])

In [8]:
july['weekend'] = july['create_dt'].apply(weekend)
august['weekend'] = august['create_dt'].apply(weekend)

In [10]:
july = july[['device_id_deidentification','create_dt','airstate.opmode','humidity','airstate.windstrength','currenttemperature','airstate.tempstate.target','airstate.opmode2','weekend']]
august = august[['device_id_deidentification','create_dt','airstate.opmode','humidity','airstate.windstrength','currenttemperature','airstate.tempstate.target','airstate.opmode2','weekend']]

In [16]:
july['id']=july['device_id_deidentification'].str[8:]
august['id']=august['device_id_deidentification'].str[8:]

/tmp/ipykernel_5971/4103349711.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  july['id']=july['device_id_deidentification'].str[8:]


In [19]:
july_id=july['id'].unique()
august_id=august['id'].unique()

In [20]:
print(len(july_id))
print(len(august_id))

2246
2045


In [25]:
common = set(july_id).intersection(august_id) # 공통 아이디 추출
len(common)

1447

In [26]:
july_=july.loc[july['id'].isin(common)]
august_=august.loc[august['id'].isin(common)]

In [29]:
summer = pd.concat([july_,august_],axis=0)

In [36]:
def make_week_df(df): 
    df_weekend = df.loc[df['weekend']==1]
    df_weekday = df.loc[df['weekend']==0]
    return df_weekend, df_weekday

In [39]:
def devide_day(df):
    df_daytime = df.loc[(df['create_dt'].dt.hour>=9)&(df['create_dt'].dt.hour<=18)]
    df_night = df.loc[(df['create_dt'].dt.hour>=19)&(df['create_dt'].dt.hour<=23)]
    df_dawn = df.loc[(df['create_dt'].dt.hour>=0)&(df['create_dt'].dt.hour<=8)]
    return df_daytime,df_night,df_dawn

In [76]:
july_weekend, july_weekday = make_week_df(july_)
august_weekend, august_weekday = make_week_df(august_)
summer_weekend, summer_weekday = make_week_df(summer)

In [78]:
july_weekend_daytime, july_weekend_night, july_weekend_dawn = devide_day(july_weekend)
july_weekday_daytime, july_weekday_night, july_weekday_dawn = devide_day(july_weekday)

august_weekend_daytime, august_weekend_night, august_weekend_dawn = devide_day(august_weekend)
august_weekday_daytime, august_weekday_night, august_weekday_dawn = devide_day(august_weekday)

summer_weekend_daytime, summer_weekend_night, summer_weekend_dawn = devide_day(summer_weekend)
summer_weekday_daytime, summer_weekday_night, summer_weekday_dawn = devide_day(summer_weekday)

In [79]:
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import bartlett
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

In [80]:
def hypothesis_test(df1,df2):

    num_col = ['currenttemperature','humidity','airstate.tempstate.target','airstate.windstrength']
    category_col = ['airstate.opmode','airstate.opmode2']


    for col in num_col:
        print(col + "-> 등분산 검정...")
        k = bartlett(df1[col],df2[col])[1] # 등분산검정
        if k < 0.05: # 두 그룹은 이분산성
            print('이분산')
            print('t test')
            if ttest_ind(df1[col],df2[col],equal_var=False)[1] < 0.05:
                print(f'두 그룹의 {col} 평균은 같지 않다')
            else:
                print(f'두 그룹의 {col} 평균은 같다')
        else:
            print('등분산')
            print('t test')
            if ttest_ind(df1[col],df2[col],equal_var=True)[1] < 0.05:
                print(f'두 그룹의 {col} 평균은 같지 않다')
            else:
                print(f'두 그룹의 {col} 평균은 같다')
                    
        print()
        
    for col in category_col:
        print(f'{col}-> proportion_ztest')
        len_df1 = len(df1)
        len_df2 = len(df2)
        count_df1 = sum(df1[col])
        count_df2 = sum(df2[col])
        
        count = np.array([count_df1,count_df2])
        nobs = np.array([len_df1,len_df2])
        
        if proportions_ztest(count=count, nobs=nobs, value=0)[1] < 0.05:
            print(f'두 그룹의 {col} 비율은 같지 않다')
        else:
            print(f'두 그룹의 {col} 비율은 같다')
            
        print()

In [71]:
hypothesis_test(july_weekday_daytime,july_weekend_daytime)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [72]:
hypothesis_test(july_weekday_night,july_weekend_night)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [73]:
hypothesis_test(july_weekday_dawn,july_weekend_dawn)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
등분산
t test
두 그룹의 airstate.tempstate.target 평균은 같다

airstate.windstrength-> 등분산 검정...
등분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [74]:
hypothesis_test(august_weekday_daytime,august_weekend_daytime)

currenttemperature-> 등분산 검정...
등분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [75]:
hypothesis_test(august_weekday_night,august_weekend_night)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [81]:
hypothesis_test(summer_weekday_daytime,summer_weekend_daytime)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [82]:
hypothesis_test(summer_weekday_night,summer_weekend_night)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [83]:
hypothesis_test(summer_weekday_dawn,summer_weekend_dawn)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
등분산
t test
두 그룹의 airstate.tempstate.target 평균은 같다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [84]:
hypothesis_test(july_,august_)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [85]:
hypothesis_test(july_,summer)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [86]:
hypothesis_test(august_,summer)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [87]:
hypothesis_test(july_weekday,july_weekend)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [88]:
hypothesis_test(august_weekday,august_weekend)

currenttemperature-> 등분산 검정...
등분산
t test
두 그룹의 currenttemperature 평균은 같지 않다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다



In [89]:
hypothesis_test(summer_weekday,summer_weekend)

currenttemperature-> 등분산 검정...
이분산
t test
두 그룹의 currenttemperature 평균은 같다

humidity-> 등분산 검정...
이분산
t test
두 그룹의 humidity 평균은 같지 않다

airstate.tempstate.target-> 등분산 검정...
이분산
t test
두 그룹의 airstate.tempstate.target 평균은 같지 않다

airstate.windstrength-> 등분산 검정...
이분산
t test
두 그룹의 airstate.windstrength 평균은 같지 않다

airstate.opmode-> proportion_ztest
두 그룹의 airstate.opmode 비율은 같지 않다

airstate.opmode2-> proportion_ztest
두 그룹의 airstate.opmode2 비율은 같지 않다

